# Milano dataset – Ubicazione consistency check & selective repair
Goal: verify that the free-text `Ubicazione` field is consistent with the structured address columns (`Tipo via`, `Descrizione via`, `Civico`, `ZD`, `Codice via`) and selectively rebuild `Ubicazione` only for rows that are clearly inconsistent (street name mismatch).

Notes:
- This notebook is organized as a reproducible pipeline (run top-to-bottom).
- We keep repairs in a new column `Ubicazione_clean` for auditability.


## 1) Imports & settings

In [20]:
import re
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 140)


## 2) Load data
Update the CSV path if needed.

In [21]:
# TODO: update path if your dataset is elsewhere
MILANO = pd.read_csv('../Dataset/Comune-di-Milano-Pubblici-esercizi(in)-2.csv',sep=';',encoding='utf-8')
MILANO.head()

,þÿTipo esercizio storico pe,Insegna,Ubicazione,Tipo via,Descrizione via,Civico,Codice via,ZD,Forma commercio,Forma commercio prev,Forma vendita,Settore storico pe,Superficie somministrazione
0,NaN,NaN,ALZ NAVIGLIO GRANDE N. 12 ; isolato:057; (z.d. 6),ALZ,NAVIGLIO GRANDE,12,5144,6,NaN,NaN,NaN,"Ristorante, trattoria, osteria;Genere Merceol....",83.0
1,NaN,NaN,ALZ NAVIGLIO GRANDE N. 44 (z.d. 6),ALZ,NAVIGLIO GRANDE,44,5144,6,NaN,NaN,NaN,Bar gastronomici e simili,26.0
2,NaN,NaN,ALZ NAVIGLIO GRANDE N. 48 (z.d. 6),ALZ,NAVIGLIO GRANDE,48,5144,6,NaN,NaN,NaN,Bar gastronomici e simili,58.0
3,NaN,NaN,ALZ NAVIGLIO GRANDE N. 8 (z.d. 6),ALZ,NAVIGLIO GRANDE,8,5144,6,NaN,NaN,NaN,"BAR CAFFÿý E SIMILI;Ristorante, trattoria, ost...",101.0
4,NaN,NaN,ALZ NAVIGLIO PAVESE N. 24 (z.d. 6),ALZ,NAVIGLIO PAVESE,24,5161,6,NaN,NaN,NaN,Bar gastronomici e simili,51.0


## 3) Keep only relevant columns (working dataframe)

In [22]:
cols = [
    'Ubicazione', 'Tipo via', 'Descrizione via', 'Civico', 'Codice via', 'ZD'
]

MILANO_COPY = MILANO.loc[:, cols].copy()
MILANO_COPY.head()

,Ubicazione,Tipo via,Descrizione via,Civico,Codice via,ZD
0,ALZ NAVIGLIO GRANDE N. 12 ; isolato:057; (z.d. 6),ALZ,NAVIGLIO GRANDE,12,5144,6
1,ALZ NAVIGLIO GRANDE N. 44 (z.d. 6),ALZ,NAVIGLIO GRANDE,44,5144,6
2,ALZ NAVIGLIO GRANDE N. 48 (z.d. 6),ALZ,NAVIGLIO GRANDE,48,5144,6
3,ALZ NAVIGLIO GRANDE N. 8 (z.d. 6),ALZ,NAVIGLIO GRANDE,8,5144,6
4,ALZ NAVIGLIO PAVESE N. 24 (z.d. 6),ALZ,NAVIGLIO PAVESE,24,5161,6


## 4) Normalize Ubicazione text (makes parsing easier)

## 5) Extract components from Ubicazione
We use small regexes (easy to debug):
- `ubi_tipo`: first 3 characters
- `ubi_desc`: text after tipo until civico marker (N / N. / num.)
- `ubi_civico`: digits after civico marker (N / N. / num.)
- `ubi_zd`: digits after `z.d.`


In [23]:
# Extracting the values from the Ubicazione field
REGEX_TIPO = r"^(.{3})"
REGEX_DESC = r"^.{3}\s+(.*?)\s+\b(?:N\.?|num\.)\b"  # stop at N / N. / num.
REGEX_CIV  = r"\b(?:N\.?|num\.)\s*0*([0-9]+)"        # digits only (optional leading zeros)
REGEX_ZD   = r"z\.d\.\s*(\d+)"                        # z.d. X

MILANO_COPY['ubi_tipo'] = MILANO_COPY['Ubicazione'].str.extract(REGEX_TIPO, expand=False)
MILANO_COPY['ubi_desc'] = MILANO_COPY['Ubicazione'].str.extract(REGEX_DESC, expand=False, flags=re.IGNORECASE)
MILANO_COPY['ubi_civico'] = MILANO_COPY['Ubicazione'].str.extract(REGEX_CIV, expand=False, flags=re.IGNORECASE)
MILANO_COPY['ubi_zd'] = MILANO_COPY['Ubicazione'].str.extract(REGEX_ZD, expand=False, flags=re.IGNORECASE)

MILANO_COPY[['Ubicazione','ubi_tipo','ubi_desc','ubi_civico','ubi_zd']].head(10)
MILANO_COPY.head(10)

,Ubicazione,Tipo via,Descrizione via,Civico,Codice via,ZD,ubi_tipo,ubi_desc,ubi_civico,ubi_zd
0,ALZ NAVIGLIO GRANDE N. 12 ; isolato:057; (z.d. 6),ALZ,NAVIGLIO GRANDE,12,5144,6,ALZ,NAVIGLIO GRANDE,12,6
1,ALZ NAVIGLIO GRANDE N. 44 (z.d. 6),ALZ,NAVIGLIO GRANDE,44,5144,6,ALZ,NAVIGLIO GRANDE,44,6
2,ALZ NAVIGLIO GRANDE N. 48 (z.d. 6),ALZ,NAVIGLIO GRANDE,48,5144,6,ALZ,NAVIGLIO GRANDE,48,6
3,ALZ NAVIGLIO GRANDE N. 8 (z.d. 6),ALZ,NAVIGLIO GRANDE,8,5144,6,ALZ,NAVIGLIO GRANDE,8,6
4,ALZ NAVIGLIO PAVESE N. 24 (z.d. 6),ALZ,NAVIGLIO PAVESE,24,5161,6,ALZ,NAVIGLIO PAVESE,24,6
5,ALZ NAVIGLIO PAVESE N. 6 (z.d. 6),ALZ,NAVIGLIO PAVESE,6,5161,6,ALZ,NAVIGLIO PAVESE,6,6
6,BST DI PORTA NUOVA N. 10 con ingr.su piazza xx...,BST,DI PORTA NUOVA,10,1062,1,BST,DI PORTA NUOVA,10,1
7,BST DI PORTA VOLTA N. 9 (z.d. 1),BST,DI PORTA VOLTA,9,1066,1,BST,DI PORTA VOLTA,9,1
8,BST DI PORTA VOLTA N. 9 (z.d. 1),BST,DI PORTA VOLTA,9,1066,1,BST,DI PORTA VOLTA,9,1
9,BST DI PORTA VOLTA num.018/a ; (z.d. 1),BST,DI PORTA VOLTA,NaN,1066,1,BST,DI PORTA VOLTA,18,1


## 6) Normalize extracted + structured fields (for fair comparisons)

In [ ]:
"""def norm_text(s: pd.Series) -> pd.Series:
    return (
        s.astype(str)
        .str.strip()
        .str.upper()
        .str.replace(r"\s+", " ", regex=True)
        .replace('NAN', np.nan)
    )


# normalize ONLY extracted values
MILANO_COPY['ubi_tipo'] = norm_text(MILANO_COPY['ubi_tipo'])
MILANO_COPY['ubi_desc'] = norm_text(MILANO_COPY['ubi_desc'])

# numeric conversions (again: only extracted ones)
MILANO_COPY['ubi_civico'] = pd.to_numeric(MILANO_COPY['ubi_civico'], errors='coerce')
MILANO_COPY['ubi_zd'] = pd.to_numeric(MILANO_COPY['ubi_zd'], errors='coerce')

# optional sanity check
MILANO_COPY[['ubi_tipo', 'Tipo via', 'ubi_desc', 'Descrizione via', 'ubi_civico', 'Civico', 'ubi_zd', 'ZD']].head()"""

## 7) Build match flags + summary
Important: for civico, a missing extracted value is not automatically a mismatch (it may be an unhandled format).

In [25]:
# match flags (True/False). If extracted is NaN, we mark match as NaN and handle separately.
MILANO_COPY['Tipo_match'] = (MILANO_COPY['ubi_tipo'] == MILANO_COPY['Tipo via'])
MILANO_COPY['Descrizione_match'] = (MILANO_COPY['ubi_desc'] == MILANO_COPY['Descrizione via'])
MILANO_COPY['Civico_match'] = (
MILANO_COPY['ubi_civico'] == MILANO_COPY['Civico'])
MILANO_COPY['ZD_match'] = (MILANO_COPY['ubi_zd'] == MILANO_COPY['ZD'])
"""summary = pd.Series({
    'rows': len(MILANO_COPY),
    'Tipo extracted NaN': MILANO_COPY['ubi_tipo_n'].isna().sum(),
    'Descrizione extracted NaN': MILANO_COPY['ubi_desc_n'].isna().sum(),
    'Civico extracted NaN': MILANO_COPY['ubi_civico_n'].isna().sum(),
    'ZD extracted NaN': MILANO_COPY['ubi_zd_n'].isna().sum(),
    'Tipo mismatches': (MILANO_COPY['Tipo_match'] == False).sum(),
    'Descrizione mismatches': (MILANO_COPY['Descrizione_match'] == False).sum(),
    'Civico mismatches': (MILANO_COPY['Civico_match'] == False).sum(),
    'ZD mismatches': (MILANO_COPY['ZD_match'] == False).sum(),
})
summary """
MILANO_COPY.head()

,Ubicazione,Tipo via,Descrizione via,Civico,Codice via,ZD,ubi_tipo,ubi_desc,ubi_civico,ubi_zd,Tipo_match,Descrizione_match,Civico_match,ZD_match
0,ALZ NAVIGLIO GRANDE N. 12 ; isolato:057; (z.d. 6),ALZ,NAVIGLIO GRANDE,12,5144,6,ALZ,NAVIGLIO GRANDE,12,6,True,True,True,False
1,ALZ NAVIGLIO GRANDE N. 44 (z.d. 6),ALZ,NAVIGLIO GRANDE,44,5144,6,ALZ,NAVIGLIO GRANDE,44,6,True,True,True,False
2,ALZ NAVIGLIO GRANDE N. 48 (z.d. 6),ALZ,NAVIGLIO GRANDE,48,5144,6,ALZ,NAVIGLIO GRANDE,48,6,True,True,True,False
3,ALZ NAVIGLIO GRANDE N. 8 (z.d. 6),ALZ,NAVIGLIO GRANDE,8,5144,6,ALZ,NAVIGLIO GRANDE,8,6,True,True,True,False
4,ALZ NAVIGLIO PAVESE N. 24 (z.d. 6),ALZ,NAVIGLIO PAVESE,24,5161,6,ALZ,NAVIGLIO PAVESE,24,6,True,True,True,False


## 8) Codice via → ZD consistency (evidence)
We check how many streets (Codice via) map to more than one ZD using the structured ZD and the extracted ZD.

In [27]:
dup_zd_struct = (MILANO_COPY.groupby('Codice via')['ZD'].nunique() > 1).sum()
dup_zd_ubi = (MILANO_COPY.groupby('Codice via')['ubi_zd'].nunique() > 1).sum()

pd.Series({
    'Codice via distinct': MILANO_COPY['Codice via'].nunique(),
    'ZD distinct': MILANO_COPY['ZD'].nunique(),
    'Codice via with >1 structured ZD': dup_zd_struct,
    'Codice via with >1 extracted ZD': dup_zd_ubi,
})

Codice via distinct                 1856
ZD distinct                            9
Codice via with >1 structured ZD      67
Codice via with >1 extracted ZD       70
dtype: int64

## 9) Selective repair policy
Conservative rule (first pass):
- If `Descrizione_match == False` (street name mismatch), rebuild Ubicazione from the structured columns.
- Otherwise, keep original Ubicazione (preserves extra notes).

Repairs are written to a new column `Ubicazione_clean`.

In [29]:
# Build canonical Ubicazione from structured fields
ubi_rebuilt = (
    MILANO_COPY['Tipo via'].astype(str).str.strip() + ' ' +
    MILANO_COPY['Descrizione via'].astype(str).str.strip() + ' N. ' +
    MILANO_COPY['Civico'].astype(str).str.strip() + ' ' +
    '(z.d. ' + MILANO_COPY['ZD'].astype(str) + ')'
).str.replace(r"\s+", " ", regex=True).str.strip()

MILANO_COPY['Ubicazione_clean'] = MILANO_COPY['Ubicazione']

mask_rebuild = (MILANO_COPY['Descrizione_match'] == False)
MILANO_COPY.loc[mask_rebuild, 'Ubicazione_clean'] = ubi_rebuilt.loc[mask_rebuild]

MILANO_COPY

,Ubicazione,Tipo via,Descrizione via,Civico,Codice via,ZD,ubi_tipo,ubi_desc,ubi_civico,ubi_zd,Tipo_match,Descrizione_match,Civico_match,ZD_match,Ubicazione_clean
0,ALZ NAVIGLIO GRANDE N. 12 ; isolato:057; (z.d. 6),ALZ,NAVIGLIO GRANDE,12,5144,6,ALZ,NAVIGLIO GRANDE,12,6,True,True,True,False,ALZ NAVIGLIO GRANDE N. 12 ; isolato:057; (z.d. 6)
1,ALZ NAVIGLIO GRANDE N. 44 (z.d. 6),ALZ,NAVIGLIO GRANDE,44,5144,6,ALZ,NAVIGLIO GRANDE,44,6,True,True,True,False,ALZ NAVIGLIO GRANDE N. 44 (z.d. 6)
2,ALZ NAVIGLIO GRANDE N. 48 (z.d. 6),ALZ,NAVIGLIO GRANDE,48,5144,6,ALZ,NAVIGLIO GRANDE,48,6,True,True,True,False,ALZ NAVIGLIO GRANDE N. 48 (z.d. 6)
3,ALZ NAVIGLIO GRANDE N. 8 (z.d. 6),ALZ,NAVIGLIO GRANDE,8,5144,6,ALZ,NAVIGLIO GRANDE,8,6,True,True,True,False,ALZ NAVIGLIO GRANDE N. 8 (z.d. 6)
4,ALZ NAVIGLIO PAVESE N. 24 (z.d. 6),ALZ,NAVIGLIO PAVESE,24,5161,6,ALZ,NAVIGLIO PAVESE,24,6,True,True,True,False,ALZ NAVIGLIO PAVESE N. 24 (z.d. 6)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6899,VLE DORIA ANDREA N. 12 ; isolato:031; accesso:...,VLE,DORIA ANDREA,12,2230,2,VLE,DORIA ANDREA,12,3,True,True,True,False,VLE DORIA ANDREA N. 12 ; isolato:031; accesso:...
6900,VIA GARIGLIANO N. 5 ; isolato:277; accesso: ac...,VIA,GARIGLIANO,5,1134,9,VIA,GARIGLIANO,5,9,True,True,True,False,VIA GARIGLIANO N. 5 ; isolato:277; accesso: ac...
6901,VIA SOTTOCORNO PASQUALE N. 4 ; isolato:014; ac...,VIA,SOTTOCORNO PASQUALE,4,3152,4,VIA,SOTTOCORNO PASQUALE,4,4,True,True,True,False,VIA SOTTOCORNO PASQUALE N. 4 ; isolato:014; ac...
6902,VIA CASTROVILLARI N. 23 ; isolato:150; accesso...,VIA,CASTROVILLARI,23,6299,7,VIA,CASTROVILLARI,23,7,True,True,True,False,VIA CASTROVILLARI N. 23 ; isolato:150; accesso...


## 10) Validate repairs (re-parse Ubicazione_clean)
Re-extract components from `Ubicazione_clean` and recompute the match flags to confirm improvement.

In [31]:
MILANO_COPY['ubi_desc2'] = MILANO_COPY['Ubicazione_clean'].str.extract(REGEX_DESC, expand=False, flags=re.IGNORECASE)
MILANO_COPY['Descrizione_match_after'] = np.where(MILANO_COPY['ubi_desc2'].notna(), MILANO_COPY['ubi_desc2'] == MILANO_COPY['Descrizione via'], np.nan)

before = (MILANO_COPY['Descrizione_match'] == False).sum()
after = (MILANO_COPY['Descrizione_match_after'] == False).sum()

pd.Series({
    'Descrizione mismatches BEFORE': before,
    'Descrizione mismatches AFTER': after,
})

Descrizione mismatches BEFORE    108
Descrizione mismatches AFTER       0
dtype: int64

## 11) Inspect repaired rows (sample)
Show a few rows that were rebuilt, with original vs cleaned.

In [33]:
MILANO_COPY.loc[mask_rebuild, ['Ubicazione', 'Ubicazione_clean', 'Tipo via', 'Descrizione via', 'Civico', 'ZD']]

,Ubicazione,Ubicazione_clean,Tipo via,Descrizione via,Civico,ZD
10,codvia 6511 parco villa scheibler; (z.d. 7),VIA CALDERA N. 21 (z.d. 7),VIA,CALDERA,21,7
11,codvia 9401 ff central; (z.d. 2),VLE MONZA N. 37 (z.d. 2),VLE,MONZA,37,2
45,CSO GARIBALDI GIUSEPPE num.051/a /a; (z.d. 1),CSO ITALIA N. 34 (z.d. 1),CSO,ITALIA,34,1
46,CSO GARIBALDI GIUSEPPE num.072/3 /3 con esposi...,VIA FOSCOLO UGO N. 4 (z.d. 1),VIA,FOSCOLO UGO,4,1
51,CSO ITALIA num.022/a a; (z.d. 1),CSO DI PORTA ROMANA N. 51 (z.d. 1),CSO,DI PORTA ROMANA,51,1
...,...,...,...,...,...,...
6287,VIA GOLA EMILIO num.016/2 ; isolato:075; acces...,VIA LODOVICO IL MORO N. 55 (z.d. 6),VIA,LODOVICO IL MORO,55,6
6290,PLE CANTORE ANTONIO num.000 ang.v.le d'annunzi...,VIA LODOVICO IL MORO N. 55 (z.d. 6),VIA,LODOVICO IL MORO,55,6
6399,VIA CASATI FELICE num.007/9 /9; isolato:104; a...,VIA CAPRANICA LUIGI N. 18 (z.d. 3),VIA,CAPRANICA LUIGI,18,3
6578,VIA TRASIMENO num.018/2 /2; isolato:139; acces...,PZA DUCA D'AOSTA N. 12 (z.d. 2),PZA,DUCA D'AOSTA,12,2


## 12) Optional export
Export the cleaned dataset or just the cleaned Ubicazione column.